In [5]:
# Modelo de algoritmo de correção de erros adaptativo

import os 
import pandas as pd
import pickle
import re
from operator import itemgetter
import random

# Abrir arquivo de dados

a_file = open("data.pkl", "rb")
data = pickle.load(a_file)

# Parametros de configuração da simulação

modulacao = 'BPSK'

sinalRuido = []
for i in range(0, 10):
    sinalRuido.append(round(random.uniform(3, 12), 1))
print('Vetor SNR:',sinalRuido, '\n\n')

codecRate = 64 # Kbps
codecPayload = 160  # Bytes
pps = 50 # milisegundos
cabecalhoIP = 40 # Bytes 
quantChamadas = 2
bandaDisponivel = 170 # Kbps

availableConfigs = []

for mod, param in data.items():
    if mod == modulacao:
        for snr, config in param.items():
            for value in sinalRuido:
                if snr == value:
                    for tupla in config:
                        rate = tupla[3]
                        if quantChamadas > 1:
                            bandaRequerida = ((((codecPayload * rate + cabecalhoIP) * 8) * pps) / 1000) * quantChamadas
                            if bandaRequerida <= bandaDisponivel:
                                availableConfigs.append(tupla)
                        else:
                            bandaRequerida = (((codecPayload * rate + cabecalhoIP) * 8) * pps) / 1000
                            if bandaRequerida <= bandaDisponivel:
                                availableConfigs.append(tupla)
                                
                    print('A banda disponível é de:', bandaDisponivel, 'Kbps')
                    
                    tabela = pd.DataFrame(availableConfigs, columns =['MOS', 'BER', 'FEC', 'RATE']) 
                    print('Configurações possiveis para modulação', mod, 'e SNR', value, 'dB:' '\n')
                    print(tabela, '\n')

                    try:
                        value = sorted(sorted(availableConfigs, key=lambda element: element[3]), 
                                       key=lambda element: element[0], reverse=True)[0]
                        print('Melhor configuração: ', value[2])
                        print('MOS=', value[0])
                        print('---------------------------------------------------------', '\n')
                    except:
                        print('Impossivel: Largura de banda insuficiente')
                    
                    list.clear(availableConfigs)
# Fechar arquivo

a_file.close()

Vetor SNR: [3.2, 9.2, 9.2, 9.4, 6.3, 5.8, 4.3, 5.7, 11.2, 10.1] 


A banda disponível é de: 170 Kbps
Configurações possiveis para modulação BPSK e SNR 3.2 dB:

       MOS       BER           FEC      RATE
0  1.00760  0.020474       No Code  1.000000
1  0.97823  0.020801  RS(255, 247)  1.032389 

Melhor configuração:  No Code
MOS= 1.0076
--------------------------------------------------------- 

A banda disponível é de: 170 Kbps
Configurações possiveis para modulação BPSK e SNR 4.3 dB:

      MOS       BER           FEC      RATE
0  1.0272  0.010169       No Code  1.000000
1  1.0289  0.010482  RS(255, 247)  1.032389 

Melhor configuração:  RS(255, 247)
MOS= 1.0289
--------------------------------------------------------- 

A banda disponível é de: 170 Kbps
Configurações possiveis para modulação BPSK e SNR 5.7 dB:

      MOS       BER           FEC      RATE
0  1.0847  0.003205       No Code  1.000000
1  1.0340  0.003076  RS(255, 247)  1.032389 

Melhor configuração:  No Code
MOS= 1.084